In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql import SparkSession
import tarfile
import os
import pyspark.sql.functions as f
from pyspark.sql.functions import col

In [2]:
# Crear Spark Session
spark = SparkSession.builder.getOrCreate()

In [3]:
%%time

# Extraer el archivo .tgz
with tarfile.open("./data/raw/yelp_dataset.tar", "r") as tar:
    tar.extractall(path="./data/ini/")

CPU times: user 7.64 s, sys: 56.8 s, total: 1min 4s
Wall time: 2min 6s


In [4]:
# Peso de review.json
file_size = os.path.getsize("./data/ini/yelp_academic_dataset_review.json")
print("Tamaño del archivo:", file_size, "bytes")

Tamaño del archivo: 5341868833 bytes


In [5]:
%%time

# Carga los archivos extraídos (review.json) en un DataFrame
df = spark.read.json("./data/ini/yelp_academic_dataset_review.json")


CPU times: user 19.1 ms, sys: 0 ns, total: 19.1 ms
Wall time: 18 s


In [7]:
%%time

# Guardar el df en parquet
df.write.parquet('review.parquet', mode="overwrite")

CPU times: user 80.1 ms, sys: 0 ns, total: 80.1 ms
Wall time: 1min 19s


In [8]:
# Peso de review.parquet
file_size = os.path.getsize("review.parquet")
print("Tamaño del archivo:", file_size, "bytes")

Tamaño del archivo: 2688 bytes


In [9]:
# Inferir el esquema inicial
initial_schema = df.schema
initial_schema

StructType([StructField('business_id', StringType(), True), StructField('cool', LongType(), True), StructField('date', StringType(), True), StructField('funny', LongType(), True), StructField('review_id', StringType(), True), StructField('stars', DoubleType(), True), StructField('text', StringType(), True), StructField('useful', LongType(), True), StructField('user_id', StringType(), True)])

In [10]:
# Definir el nuevo esquema
optimized_schema = StructType([
    StructField("review_id", StringType(), nullable=True),
    StructField("user_id", StringType(), nullable=True),
    StructField("business_id", StringType(), nullable=True),
    StructField("stars", IntegerType(), nullable=True),
    StructField("date", DateType(), nullable=True),
    StructField("text", StringType(), nullable=True),
    StructField("useful", IntegerType(), nullable=True),
    StructField("funny", IntegerType(), nullable=True),
    StructField("cool", IntegerType(), nullable=True)
])

In [11]:
%%time

# Aplicar el esquema optimizado al DataFrame
df_optimized = spark.read.schema(optimized_schema).json("./data/ini/yelp_academic_dataset_review.json")

CPU times: user 7.73 ms, sys: 0 ns, total: 7.73 ms
Wall time: 51.9 ms


In [12]:
%%time

# Guardar el df_optimized en parquet
df_optimized.write.parquet('review_op.parquet', mode="overwrite")

CPU times: user 97.6 ms, sys: 0 ns, total: 97.6 ms
Wall time: 1min 52s


In [13]:
# Peso del parquet review_op.parquet
file_size = os.path.getsize("review_op.parquet")
print("Tamaño del archivo:", file_size, "bytes")

Tamaño del archivo: 2688 bytes


In [31]:
%%time

# Carga los archivos extraídos business.json en un DataFrame
df_bus = spark.read.json("./data/ini/yelp_academic_dataset_business.json")
df_bus = df_bus.drop("stars")

CPU times: user 9.86 ms, sys: 0 ns, total: 9.86 ms
Wall time: 1.11 s


Una vez realizados los pasos anteriores crearemos el dataframe "df_optimized_v2" que contiene la fecha
desglozada por año y mes. Además haremos un join de ese dataframe con con df_bus.

In [16]:
%%time

# Extraer columnas de interés de df_optimized
df_optimized_v2 = (
    df_optimized
     .withColumn("year", f.year(f.col("date")))
     .withColumn("month", f.month(f.col("date")))
     .select("review_id", "user_id", "business_id", "stars", "year", "month", "date", "text", "useful", "funny", "cool")
)

CPU times: user 17.6 ms, sys: 0 ns, total: 17.6 ms
Wall time: 170 ms


In [32]:
%%time

# Realizar inner join en la columna 'business_id'
joined_df = df_bus.join(df_optimized_v2, "business_id", "inner")

CPU times: user 7.56 ms, sys: 0 ns, total: 7.56 ms
Wall time: 42.3 ms


In [35]:
%%time

# Parquet particionado por fecha
(df_optimized_v2
     .write.parquet(
         "op_df_v1", 
         mode="overwrite", 
         partitionBy=["year", "month"],
         compression="gzip"
     )
)

CPU times: user 270 ms, sys: 0 ns, total: 270 ms
Wall time: 5min 9s


In [37]:
%%time

# Parquet particionado por ciudad
(joined_df
     .write.parquet(
         "jo_df_v1", 
         mode="overwrite", 
         partitionBy=["city"],
         compression="gzip"
     )
)

Py4JJavaError: An error occurred while calling o321.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 28.0 failed 1 times, most recent failure: Lost task 3.0 in stage 28.0 (TID 410) (3fd5f18d14d7 executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/home/jovyan/work/demo/09/jo_df_v1.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRead

In [36]:
%%time

# Parquet particionado por ciudad y fecha
(joined_df
     .write.parquet(
         "jo_df_v2", 
         mode="overwrite", 
         partitionBy=["city", "year", "month"],
         compression="gzip"
     )
)

Py4JJavaError: An error occurred while calling o312.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 23.0 failed 1 times, most recent failure: Lost task 2.0 in stage 23.0 (TID 360) (3fd5f18d14d7 executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/home/jovyan/work/demo/09/joined_df_v2.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOr